<a href="https://colab.research.google.com/github/Matte920/DeepLearning_PracticaFinal/blob/main/Hito3_LateFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow import keras
from tensorflow.keras import layers



df_meta = pd.read_csv("/content/HAM10000_metadata.csv")

df_img = pd.read_csv("/content/hnmist_28_28_RGB.csv")

print("Shape metadata:", df_meta.shape)
print("Shape imágenes:", df_img.shape)



y_str = df_meta["dx"].values
le = LabelEncoder()
y_int = le.fit_transform(y_str)
num_classes = len(le.classes_)
y_oh = keras.utils.to_categorical(y_int, num_classes=num_classes)



X_tab = df_meta[["sex", "age", "localization"]].copy()

X_tab["sex"] = X_tab["sex"].fillna("unknown")
X_tab["localization"] = X_tab["localization"].fillna("unknown")
X_tab["age"] = X_tab["age"].fillna(X_tab["age"].median())


X_tab = pd.get_dummies(X_tab, columns=["sex", "localization"])

X_tab = X_tab.astype("float32")


input_dim_tab = X_tab.shape[1]



X_img = df_img.values.astype("float32")


X_img = X_img / 255.0

num_pixels = X_img.shape[1]
print("Num. píxeles por imagen:", num_pixels)

assert num_pixels == 28 * 28 * 3, "Número de píxeles inesperado."


X_img = X_img.reshape(-1, 28, 28, 3)
input_shape_img = (28, 28, 3)

print("Shape X_img (después reshape):", X_img.shape)


indices = np.arange(len(y_int))

idx_train, idx_test = train_test_split(
    indices,
    test_size=0.2,
    random_state=42,
    stratify=y_int
)

X_train_tab = X_tab.values[idx_train]
X_test_tab  = X_tab.values[idx_test]

X_train_img = X_img[idx_train]
X_test_img  = X_img[idx_test]

y_train = y_oh[idx_train]
y_test  = y_oh[idx_test]


scaler = StandardScaler()
X_train_tab[:, 0:1] = scaler.fit_transform(X_train_tab[:, 0:1])
X_test_tab[:, 0:1] = scaler.transform(X_test_tab[:, 0:1])

print("Shape X_train_tab:", X_train_tab.shape)
print("Shape X_train_img:", X_train_img.shape)
print("Shape y_train:", y_train.shape)


input_tab = keras.Input(shape=(input_dim_tab,), name="tabular_input")

t = layers.Dense(64, activation="relu")(input_tab)
t = layers.Dropout(0.3)(t)
t = layers.Dense(32, activation="relu")(t)
t = layers.Dropout(0.3)(t)
out_tab = layers.Dense(num_classes, activation="softmax", name="tabular_output")(t)

model_tab = keras.Model(inputs=input_tab, outputs=out_tab, name="modelo_1D_tabular")

model_tab.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_tab = model_tab.fit(
    X_train_tab,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

loss_tab_test, acc_tab_test = model_tab.evaluate(X_test_tab, y_test, verbose=0)
print("Test accuracy modelo 1D (tabular):", acc_tab_test)



input_img = keras.Input(shape=input_shape_img, name="imagen_input")

x = layers.Conv2D(32, (3, 3), activation="relu", padding="same")(input_img)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation="relu", padding="same")(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)
out_img = layers.Dense(num_classes, activation="softmax", name="imagen_output")(x)

model_2d = keras.Model(inputs=input_img, outputs=out_img, name="modelo_2D_imagenes")

model_2d.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_2d = model_2d.fit(
    X_train_img,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    verbose=1
)

loss_img_test, acc_img_test = model_2d.evaluate(X_test_img, y_test, verbose=0)
print("Test accuracy modelo 2D (imágenes):", acc_img_test)




pred_tab_train = model_tab.predict(X_train_tab)
pred_tab_test  = model_tab.predict(X_test_tab)

pred_img_train = model_2d.predict(X_train_img)
pred_img_test  = model_2d.predict(X_test_img)

print("Shape pred_tab_train:", pred_tab_train.shape)
print("Shape pred_img_train:", pred_img_train.shape)


Z_train = np.concatenate([pred_tab_train, pred_img_train], axis=1)
Z_test  = np.concatenate([pred_tab_test,  pred_img_test],  axis=1)

print("Shape Z_train (late-fusion):", Z_train.shape)


input_fusion = keras.Input(shape=(2 * num_classes,), name="fusion_input")

f = layers.Dense(32, activation="relu")(input_fusion)
f = layers.Dropout(0.3)(f)
out_fusion = layers.Dense(num_classes, activation="softmax", name="fusion_output")(f)

model_fusion = keras.Model(inputs=input_fusion, outputs=out_fusion, name="modelo_late_fusion")

model_fusion.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fusion = model_fusion.fit(
    Z_train,
    y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

loss_fusion_test, acc_fusion_test = model_fusion.evaluate(Z_test, y_test, verbose=0)
print("Test accuracy (estrategia LATE-FUSION – HITO 3):", acc_fusion_test)

Shape metadata: (10015, 7)
Shape imágenes: (10015, 2352)
Num. píxeles por imagen: 2352
Shape X_img (después reshape): (10015, 28, 28, 3)
Shape X_train_tab: (8012, 19)
Shape X_train_img: (8012, 28, 28, 3)
Shape y_train: (8012, 7)
Epoch 1/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.4549 - loss: 1.6141 - val_accuracy: 0.6862 - val_loss: 0.9692
Epoch 2/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6803 - loss: 1.0484 - val_accuracy: 0.7018 - val_loss: 0.9241
Epoch 3/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6904 - loss: 1.0072 - val_accuracy: 0.7018 - val_loss: 0.9051
Epoch 4/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6890 - loss: 1.0035 - val_accuracy: 0.6987 - val_loss: 0.8976
Epoch 5/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6886 - loss: 0.9887 - val_accuracy: 0.6993 - val_loss: 0.8948
Epoch 6/10
101/101 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7000 - loss: 0.9554 - val_accuracy: 0.6981 - val_loss: 0.8925
Epo

El modelo que combina las predicciones 1D y 2D obtiene una accuracy ≈0.68.
Confirma que fusionar ambas fuentes mejora respecto al modelo tabular.